## Overview:

This notebook processes and analyzes model test result JSON files to extract unique model responses, their scores and frequencies, and save the results for different context types. It includes functions to:

- Load and process JSON files for a given context and ID.
- Extract unique responses, their scores, and frequencies.
- Save the processed data for all context types into a single JSON file for a specified ID.

Additionally, it demonstrates saving the results for a specific ID as an example.

### Helper Functions


Load and Process JSON Files:

In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
import tqdm
from dotenv import load_dotenv
import os
import time
from groq import Groq



def __get_unique_model_responses(id: int, context_type: str, with_misleading: bool = False) -> list:
    """
    Load and process model test result JSON files for a given context and ID.

    Returns a list of unique model responses with their:
    - score (from first occurrence)
    - frequency (how many times that response appeared)
    
    Sorted by descending score.
    """
    model_name = 'llama-2-7b-80k'
    base_dir = Path('../results/graph')
    
    suffix = '_misleading' if with_misleading else ''
    directory = base_dir / f'{model_name}_id_{id}_{context_type}{suffix}'

    # Collect all responses and scores
    data = [
        json.load(open(directory / file, encoding='utf-8'))
        for file in os.listdir(directory) if file.endswith('.json')
    ]
    
    df = pd.DataFrame(data)[['model_response', 'needle', 'context_length', 'depth_percent']]
    question_df = pd.read_excel("../data_generation/FreshQADataset_with_misleading.xlsx")
    try:
        question = question_df.set_index("id").at[id, "question"]
        df['question'] = question
        
    except KeyError:
        raise ValueError(f"No question found for id: {id}")

    # Compute frequency
    freq = df['model_response'].value_counts().rename('frequency')

    # Drop duplicates but keep first score
    # df = df.drop_duplicates(subset='model_response')

    # Merge with frequency
    df = df.merge(freq, left_on='model_response', right_index=True)

    # Sort by score descending
    # df = df.sort_values(by='score', ascending=False).reset_index(drop=True)

    return df.to_dict(orient='records')


Save Processed Results:

In [10]:
def save_all_contexts_for_id(id: int) -> None:
    """
    Gather results from all 4 context types and save them in a single JSON file for the given id.

    File will be saved as: unique_responses/{id}.json
    """
    save_dir = Path('unique_responses')
    save_dir.mkdir(parents=True, exist_ok=True)

    all_data = {
        'relevant': __get_unique_model_responses(id, 'relevant', with_misleading=False),
        'relevant_misleading': __get_unique_model_responses(id, 'relevant', with_misleading=True),
        'irrelevant': __get_unique_model_responses(id, 'irrelevant', with_misleading=False),
        'irrelevant_misleading': __get_unique_model_responses(id, 'irrelevant', with_misleading=True),
    }

    save_path = save_dir / f'{id}.json'
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)

    print(f"Saved all context responses for ID {id} to {save_path}")

### Run Analysis:

In [11]:
id_val = 44

In [12]:
save_all_contexts_for_id(id_val)

KeyError: "None of [Index(['model_response', 'needle', 'context_length', 'depth_percent'], dtype='object')] are in the [columns]"

In [8]:
__get_unique_model_responses(id_val, 'relevant', with_misleading=False)

[{'model_response': 'No, the FIFA World Cup will not be hosted this year.',
  'score': 68.16167831420898,
  'needle': "There won't be a FIFA World Cup this year.",
  'frequency': 10},
 {'model_response': 'No, the FIFA World Cup will not be hosted this year. The FIFA World Cup is a major international football tournament that is held every four years. The tournament is organized by FIFA, the international governing body of football, and is contested by',
  'score': 64.01805877685547,
  'needle': "There won't be a FIFA World Cup this year.",
  'frequency': 1},
 {'model_response': "No, the FIFA World Cup will not be hosted this year. The FIFA World Cup is a major international football tournament that is held every four years. The tournament is organized by the Fédération Internationale de Football Association (FIFA), the sport'",
  'score': 63.443970680236816,
  'needle': "There won't be a FIFA World Cup this year.",
  'frequency': 1},
 {'model_response': 'No, the FIFA World Cup will not

In [ ]:
def build_rewrite_prompt(model_response, real_needle, question):
    return (
        f"""You are evaluating a model-generated response for hallucination analysis in a long-context retrieval setting.

            You are given the following:

            Question: {question}

            The correct answer (real needle): {real_needle}

            A model-generated response: {model_response}

            Your task is to classify the model's response into one of the following categories:

            1. Correct Retrieval: the model answered the question correctly

            2. Excess Incorrect Retrieval: the model answered the question correctly but it also went on to output other text that was either completely irrelevant or invalid with regards to the question.

            3. Irrelevant Retrieval: the model did not answer the question, it went on to output other text

            4. Contradictory Retrieval: the model answered the question incorrectly

            Examples:

            Question: Who is the President of the United States? Needle: The President of the United States is Donald Trump.

            1: The President of the United States is Donald Trump. (Correct Retrieval)

            2: The President of the United States is Donald Trump. The affirmation was made in the Declaration of Independence, which was written predominantly by Thomas Jefferson in 1776. (Excess Incorrect Retrieval)

            3: The President of the United States is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. (Irrelevant Retrieval)

            4: Joe Biden is the President of the United States. (Contradictory Retrieval)

            Return your output in this exact JSON format: {{"model_response": "<text>", "needle": "<real_needle>", "hallucination_type": "<Correct Retrieval | Excess Incorrect Retrieval | Irrelevant Retrieval | Contradictory Retrieval>"}}"""
    )

In [10]:
# Load .env and API key
load_dotenv()
groq_api_key = os.getenv("snlp_api_key")

# Initialize Groq client
client = Groq(api_key=groq_api_key)

def rewrite_question_and_needle(question, needle):
    while True:
        try:
            prompt = build_rewrite_prompt(question, needle)
            completion = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=512,
                top_p=1.0,
                stream=False,
                response_format={"type": "json_object"},
                stop=None,
            )
            content = completion.choices[0].message.content
            data = json.loads(content)
            return data["hallucination_type"]
        except Exception as e:
            print(f"Error: {e}")
            print("Retrying now...")


In [11]:

with open(f"unique_responses/{id_val}.json", "r") as f:
    data = json.load(f)
    
rows = []
for category, entries in data.items():
    for entry in entries:
        entry["category"] = category
        rows.append(entry)

df = pd.DataFrame(rows)
print(df)


                                       model_response      score  \
0   No, the FIFA World Cup will not be hosted this...  68.161678   
1   No, the FIFA World Cup will not be hosted this...  64.018059   
2   No, the FIFA World Cup will not be hosted this...  63.443971   
3   No, the FIFA World Cup will not be hosted this...  61.143923   
4   No. The FIFA World Cup is a quadrennial event....  52.484453   
5   No. The 1942 and 1946 competitions, which Germ...  42.264229   
6                                                 No.  11.441657   
7      No, there won't be a FIFA World Cup this year.  84.071064   
8   No, the FIFA World Cup will not be hosted this...  68.161678   
9   No, the FIFA World Cup will not be hosted this...  64.754856   
10  No, the FIFA World Cup will not be hosted this...  60.974330   
11                                                No.  11.441657   
12                                      No, it won't.  10.532115   
13  No, the FIFA World Cup will not be held this

In [ ]:
from tqdm import tqdm
hallucination_types = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Refining Q&A pairs"):
    mr, n, q = row.get("model_response", ""), row.get("needle", ""), row.get("question", "")
    # if s > 50:
    #     current_hallucination_type = "No Hallucination"
    
    # else:
    if idx > 0 and idx % 5 == 0:
        print("⏳ Rate limit pause: sleeping for 15 seconds...")
        time.sleep(15)
        
    current_hallucination_type = rewrite_question_and_needle(mr, n, q)
    hallucination_types.append(current_hallucination_type)

# Update the DataFrame
df['hallucination_type'] = hallucination_types

Refining Q&A pairs:   0%|          | 0/23 [00:00<?, ?it/s]

⏳ Rate limit pause: sleeping for 15 seconds...


Refining Q&A pairs:  74%|███████▍  | 17/23 [00:17<00:03,  1.92it/s]

⏳ Rate limit pause: sleeping for 15 seconds...


Refining Q&A pairs: 100%|██████████| 23/23 [00:33<00:00,  1.45s/it]


In [15]:
# Group by category and convert each group to a list of dicts
json_output = {
    category: group.drop(columns="category").to_dict(orient="records")
    for category, group in df.groupby("category")
}

output_dir = "updated_unique_responses"
file_id = str(id_val)  # or whatever variable you're using
output_path = os.path.join(output_dir, f"{file_id}.json")

# Create the folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Write the JSON file into the folder
with open(output_path, "w") as f:
    json.dump(json_output, f, indent=4)
